In [ ]:
import datetime as dt

import numpy as np

from cryosat_utils import *
%matplotlib inline

In [ ]:
subplot_size=5

sit_names = ['cs2', 'inp', 'lor', 'hir']
p_vec = (10,20,30)

date_delta0, date_delta1 = -2, 2

In [ ]:
date0 = dt.datetime(2021,1,5)
cs2_files, sit_inp_name, sit_lor_name, sit_hir_name = get_filenames(date0, date_delta0, date_delta1)
odf = read_cs2_orbits(cs2_files)
plot_cs2_orbits_hist(odf)
sit_inp, sit_lor, sit_hir = get_sit_2d(sit_inp_name, sit_lor_name, sit_hir_name)
rho_ice_new, rho_snow_new  = fit_snow_ice_rho(odf, sit_inp, plot=True)
odf_masked = get_masked_odf(odf, rho_ice_new, rho_snow_new)
c, r, h = get_smoothed_resampled_orbits(odf_masked, smth_time='12s', resample_time='6s', min_periods=5, t0=None, t1=None)
plot_orbit_mean(c, r, h, [sit_inp, sit_lor, sit_hir])
sit_cs2 = rasterize_cs2(c, r, h, sit_inp, ws=2)

In [ ]:
ano = {}
for arr, name in zip([sit_cs2, sit_inp, sit_lor, sit_hir], sit_names):
    print(name)
    ano[name] = compute_anomaly(arr)

In [ ]:
titles = ['CS2', 'CS2SMOS', 'CNN-LR', 'CNN-HR']

print('FIELDS')
arrays = [sit_cs2, sit_inp, sit_lor, sit_hir]
plot_maps(arrays, titles, clim=(0, 3), subplot_size=7, step=1)

print('ANOMALY')
arrays = [ano[j][1] for j in sit_names]
plot_maps(arrays, titles, clim=(-0.2, 0), subplot_size=7, step=1)

In [ ]:
ppp = {}
for arr, name in zip([sit_cs2, sit_inp, sit_lor, sit_hir], sit_names):
    print(name)
    ppp[name] = compute_precentiles(ano[name][1], p_vec, stp=5)
    
for p in p_vec:
    print(f'ANOMALY P{p}')
    arrays = [ppp[i][p] for i in sit_names]
    plot_maps(arrays, titles, clim=(-0.2, 0), subplot_size=7, step=1)

In [ ]:
arrays = [ano[i][1] for i in sit_names]
print('Common mask')
plot_qq(arrays, titles)
print('Individual mask')
plot_qq(arrays, titles, common_mask=False)

In [ ]:
c2, r2, h2 = get_smoothed_resampled_orbits(odf_masked, smth_time='30s', resample_time='6s', min_periods=5, t0=None, t1=None)
plot_orbit_mean(c2, r2, h2, [sit_inp, sit_lor, sit_hir])

In [ ]:
sit_cs22 = rasterize_cs2(c2, r2, h2, sit_inp, ws=5)
sit_names2 = ['cs2', 'cs22', 'inp', 'lor', 'hir']

ano2 = {}
for arr, name in zip([sit_cs2, sit_cs22, sit_inp, sit_lor, sit_hir], sit_names2):
    print(name)
    ano2[name] = compute_anomaly(arr)

p_vec2 = (10,20,)
ppp2 = {}
for arr, name in zip([sit_cs2, sit_cs22, sit_inp, sit_lor, sit_hir], sit_names2):
    print(name)
    ppp2[name] = compute_precentiles(ano2[name][1], p_vec2, stp=5)

In [ ]:
titles2 = ['CS2_12km', 'CS2_30km', 'CS2SMOS', 'CNN-LR', 'CNN-HR']
for p in p_vec2:
    print(f'ANOMALY P{p}')
    arrays = [ppp2[i][p] for i in sit_names2]
    plot_maps(arrays, titles2, clim=(-0.2, 0), subplot_size=7, step=1)
    
    
arrays = [ano2[i][1] for i in sit_names2]
print('Common mask')
plot_qq(arrays, titles2)
print('Individual mask')
plot_qq(arrays, titles2, common_mask=False)

In [ ]:
colors = ['b', 'g']
labels = ['CS2SMOS', 'CNN-HI-RES']
arrays = [sit_inp, sit_hir]
plot_orbit_anomaly_quantile(odf_masked, arrays, colors, labels)